In [1]:
import pandas as pd
import plotly.express as px

In [2]:
kuka_output = pd.read_excel('KukaOutput-ICNAP.xlsx', engine='openpyxl')
kuka_output.head()

,KukaRobotId,Product,StartDate,EndDate
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59


In [3]:
kuka_output['EndDate'] = pd.to_datetime(kuka_output['EndDate'])

In [4]:
t0 = kuka_output['EndDate'].iloc[0]
kuka_output['EndDate min'] = (kuka_output['EndDate'] - t0).astype('timedelta64[m]')

In [5]:
kuka_output

,KukaRobotId,Product,StartDate,EndDate,EndDate min
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0
...,...,...,...,...,...
526,82403,Rotor A1,2022-02-28 20:14:35,2022-02-28 23:24:19,40222.0
527,82402,Rotor B3,2022-02-28 23:24:00,2022-03-01 01:14:49,40333.0
528,82403,Rotor A1,2022-02-28 23:29:09,2022-03-01 02:45:57,40424.0
529,68050,Rotor C1,2022-02-28 23:46:24,2022-03-01 04:13:09,40511.0


In [6]:
programs = pd.read_excel('OvenPrograms-ICNAP.xlsx', engine='openpyxl')
programs

,program name,product type,target temp (C),program duration (min),cooling temp (C),time to cooling (min)
0,A01 Kuk,Rotor A0,319.55085,260,42.60678,60
1,A01 Kuk,Rotor A1,319.55085,260,42.60678,60
2,A23 Kuk,Rotor A2,319.55085,390,42.60678,60
3,A23 Kuk,Rotor A3,319.55085,390,42.60678,60
4,A456 Kuk,Rotor A4,319.55085,400,42.60678,60
5,A456 Kuk,Rotor A5,319.55085,400,42.60678,60
6,A456 Kuk,Rotor A6,319.55085,400,42.60678,60
7,B2 Kuk,Rotor B2,326.65198,240,71.01130,60
8,B3 Kuk,Rotor B3,326.65198,240,71.01130,60
9,B456+B6 Kuk,Rotor B4,326.65198,390,71.01130,60


In [7]:
ovens = []
names = []
ovens_index = []
start_times = []
end_times = []

for i in range(kuka_output.shape[0]):
    product = kuka_output['Product'].iloc[i]
    time = kuka_output['EndDate min'].iloc[i]
    duration = programs[programs['product type']==product]['program duration (min)'].values[0] + 60
    if i < 6:
        ovens.append([time+duration])
        names.append([product])
        ovens_index.append(i)
        start_times.append(time)
        end_times.append(time+duration)

    else:
        kt = [oven[-1]-time for oven in ovens]
        index = kt.index(min(kt))
        names[index].append(product)
        ovens_index.append(index)
        if min(kt) <= 0:
            start_times.append(time)
            end_times.append(time+duration)
            ovens[index].append(time+duration)

        else:
            start_times.append(ovens[index][-1])
            end_times.append(ovens[index][-1]+duration)
            ovens[index].append(ovens[index][-1]+duration)


kuka_output['oven_number'] = ovens_index
kuka_output['start_job'] = start_times
kuka_output['finish_job'] = end_times
kuka_output.head(20)

,KukaRobotId,Product,StartDate,EndDate,EndDate min,oven_number,start_job,finish_job
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0,0,0.0,450.0
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0,1,1.0,451.0
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0,2,127.0,587.0
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0,3,136.0,586.0
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0,4,199.0,649.0
5,68050,Rotor A3,2022-02-01 04:21:22,2022-02-01 06:40:31,338.0,5,338.0,788.0
6,68050,Rotor C1,2022-02-01 06:45:10,2022-02-01 06:45:43,343.0,0,450.0,690.0
7,82403,Rotor A2,2022-02-01 03:18:34,2022-02-01 07:07:44,365.0,1,451.0,901.0
8,82402,Rotor B3,2022-02-01 03:09:33,2022-02-01 09:46:59,525.0,3,586.0,886.0
9,82402,Rotor B3,2022-02-01 09:47:05,2022-02-01 09:53:56,532.0,2,587.0,887.0


In [15]:
kuka_output['start_job_date'] = t0 + pd.to_timedelta(kuka_output['start_job'], unit='m')
kuka_output['finish_job_date'] = t0 + pd.to_timedelta(kuka_output['finish_job'], unit='m')
kuka_output.head(10)

,index,KukaRobotId,Product,StartDate,EndDate,EndDate min,oven_number,start_job,finish_job,start_job_date,finish_job_date
0,0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0,0,0.0,450.0,2022-02-01 01:01:47,2022-02-01 08:31:47
1,1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0,1,1.0,451.0,2022-02-01 01:02:47,2022-02-01 08:32:47
2,2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0,2,127.0,587.0,2022-02-01 03:08:47,2022-02-01 10:48:47
3,3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0,3,136.0,586.0,2022-02-01 03:17:47,2022-02-01 10:47:47
4,4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0,4,199.0,649.0,2022-02-01 04:20:47,2022-02-01 11:50:47
5,5,68050,Rotor A3,2022-02-01 04:21:22,2022-02-01 06:40:31,338.0,5,338.0,788.0,2022-02-01 06:39:47,2022-02-01 14:09:47
6,6,68050,Rotor C1,2022-02-01 06:45:10,2022-02-01 06:45:43,343.0,0,450.0,690.0,2022-02-01 08:31:47,2022-02-01 12:31:47
7,7,82403,Rotor A2,2022-02-01 03:18:34,2022-02-01 07:07:44,365.0,1,451.0,901.0,2022-02-01 08:32:47,2022-02-01 16:02:47
8,8,82402,Rotor B3,2022-02-01 03:09:33,2022-02-01 09:46:59,525.0,3,586.0,886.0,2022-02-01 10:47:47,2022-02-01 15:47:47
9,9,82402,Rotor B3,2022-02-01 09:47:05,2022-02-01 09:53:56,532.0,2,587.0,887.0,2022-02-01 10:48:47,2022-02-01 15:48:47


In [13]:
kuka_output.reset_index(inplace=True)

In [14]:
fig = px.timeline(kuka_output, x_start="start_job_date", x_end="finish_job_date", y="oven_number", color="Product", text="index")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [10]:
kuka_output.iloc[:9]

,KukaRobotId,Product,StartDate,EndDate,EndDate min,oven_number,start_job,finish_job,start_job_date,finish_job_date
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0,0,0.0,450.0,2022-02-01 01:01:47,2022-02-01 08:31:47
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0,1,1.0,451.0,2022-02-01 01:02:47,2022-02-01 08:32:47
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0,2,127.0,587.0,2022-02-01 03:08:47,2022-02-01 10:48:47
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0,3,136.0,586.0,2022-02-01 03:17:47,2022-02-01 10:47:47
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0,4,199.0,649.0,2022-02-01 04:20:47,2022-02-01 11:50:47
5,68050,Rotor A3,2022-02-01 04:21:22,2022-02-01 06:40:31,338.0,5,338.0,788.0,2022-02-01 06:39:47,2022-02-01 14:09:47
6,68050,Rotor C1,2022-02-01 06:45:10,2022-02-01 06:45:43,343.0,0,450.0,690.0,2022-02-01 08:31:47,2022-02-01 12:31:47
7,82403,Rotor A2,2022-02-01 03:18:34,2022-02-01 07:07:44,365.0,1,451.0,901.0,2022-02-01 08:32:47,2022-02-01 16:02:47
8,82402,Rotor B3,2022-02-01 03:09:33,2022-02-01 09:46:59,525.0,3,586.0,886.0,2022-02-01 10:47:47,2022-02-01 15:47:47


In [16]:
kuka_output.tail()

,index,KukaRobotId,Product,StartDate,EndDate,EndDate min,oven_number,start_job,finish_job,start_job_date,finish_job_date
526,526,82403,Rotor A1,2022-02-28 20:14:35,2022-02-28 23:24:19,40222.0,0,40561.0,40881.0,2022-03-01 05:02:47,2022-03-01 10:22:47
527,527,82402,Rotor B3,2022-02-28 23:24:00,2022-03-01 01:14:49,40333.0,4,40705.0,41005.0,2022-03-01 07:26:47,2022-03-01 12:26:47
528,528,82403,Rotor A1,2022-02-28 23:29:09,2022-03-01 02:45:57,40424.0,5,40719.0,41039.0,2022-03-01 07:40:47,2022-03-01 13:00:47
529,529,68050,Rotor C1,2022-02-28 23:46:24,2022-03-01 04:13:09,40511.0,2,40758.0,40998.0,2022-03-01 08:19:47,2022-03-01 12:19:47
530,530,82401,Rotor A5,2022-02-28 12:27:06,2022-03-01 06:17:10,40635.0,3,40837.0,41297.0,2022-03-01 09:38:47,2022-03-01 17:18:47
